In [1]:
#Connecting to google drive
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Colab\ Notebooks/RSChallenge/RSLib
%ls

/content/gdrive/MyDrive/Colab Notebooks/RSChallenge/RSLib
RecSys_Course_AT_PoliMi-master/


In [3]:
!ls */*.py

RecSys_Course_AT_PoliMi-master/cython_example_FunkSVD.py
RecSys_Course_AT_PoliMi-master/cython_example_SLIM_MSE.py
RecSys_Course_AT_PoliMi-master/run_all_algorithms.py
RecSys_Course_AT_PoliMi-master/run_compile_all_cython.py
RecSys_Course_AT_PoliMi-master/run_hyperparameter_search.py
RecSys_Course_AT_PoliMi-master/run_test_recommenders.py


In [4]:
#Importing course's repositories
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/RSChallenge/RSLib/RecSys_Course_AT_PoliMi-master')

In [5]:
#To use skopt
! pip install --quiet lightgbm==2.2.3 scikit-optimize==0.9.0 neptune-client==0.4.132 neptune-contrib[monitoring]==0.25.0

     |████████████████████████████████| 100 kB 4.9 MB/s 
     |████████████████████████████████| 110 kB 43.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 829 kB 31.7 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 180 kB 56.5 MB/s 
     |████████████████████████████████| 106 kB 49.7 MB/s 
     |████████████████████████████████| 175 kB 58.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 99 kB 9.3 MB/s 
     |████████████████████████████████| 130 kB 48.1 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 564 kB 33.4 MB/s 


In [6]:
#Importing libraries
import pandas as pd 
import scipy.sparse as sps
import numpy as np
import time
import matplotlib.pyplot as pyplot
import os
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

print("DEBUG")

DEBUG


In [7]:
%cd ..
%ls

/content/gdrive/MyDrive/Colab Notebooks/RSChallenge
HYBRID.ipynb  Results/  RSData/  RSLib/  RSLib2/


In [8]:
#Loading dataframes
URM_dataframe = pd.read_csv('RSData/data_train.csv')
ICM_genre_dataframe=pd.read_csv('RSData/data_ICM_genre.csv')
ICM_subgenre_dataframe=pd.read_csv('RSData/data_ICM_subgenre.csv')
ICM_event_dataframe=pd.read_csv('RSData/data_ICM_event.csv')
ICM_channel_dataframe=pd.read_csv('RSData/data_ICM_channel.csv')

In [9]:
#Column Renaming
URM_dataframe.columns = ['User','Item','Interaction']
ICM_genre_dataframe.columns = ['Item','Feature','Val']
ICM_subgenre_dataframe.columns = ['Item','Feature','Val']
ICM_event_dataframe.columns = ['Item','Feature','Val']
ICM_channel_dataframe.columns = ['Item','Feature','Val']

In [10]:
#Converting datatype
URM_dataframe['Interaction'] = URM_dataframe['Interaction'].astype(int)
ICM_genre_dataframe['Val'] = ICM_genre_dataframe['Val'].astype(int)
ICM_subgenre_dataframe['Val'] = ICM_subgenre_dataframe['Val'].astype(int)
ICM_event_dataframe['Val'] = ICM_event_dataframe['Val'].astype(int)
ICM_channel_dataframe['Val'] = ICM_channel_dataframe['Val'].astype(int)

In [11]:
#Printing stats to see if everything went well
userID_unique = URM_dataframe["User"].unique()
itemID_unique = URM_dataframe["Item"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_dataframe)

#Printing some stats
print("Number of users: " + str(n_users))
print("Number of items: " + str(n_items))
print("Number of interactions: " + str(n_interactions))

Number of users: 13650
Number of items: 18059
Number of interactions: 5285664


In [12]:
#Checking if re-assignment of ID's is needed
print('The maxID for user {}, The maxID for item {}'.format(max(URM_dataframe.User),max(URM_dataframe.Item)))

The maxID for user 13649, The maxID for item 18058


Re-assignment not needed

In [13]:
#Converting the URM into a sparse matrix
URM_sparse = sps.coo_matrix((URM_dataframe['Interaction'].values,(URM_dataframe['User'].values,URM_dataframe['Item'].values)))
URM_csr = URM_sparse.tocsr()
URM_csr

<13650x18059 sparse matrix of type '<class 'numpy.longlong'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [14]:
#Splitting data into training and validation set (Test set is the one on Kaggle)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


This requires a lot of time, let's first try with some random values ...

In [15]:
recommender = SLIMElasticNetRecommender(URM_train)

In [16]:
recommender.fit(l1_ratio = 0.0002204448537910578, alpha = 0.079724114437381, topK = 3100) # 47.33%

SLIMElasticNetRecommender: Processed 1478 ( 8.2%) in 5.00 min. Items per second: 4.92
SLIMElasticNetRecommender: Processed 3027 (16.8%) in 10.01 min. Items per second: 5.04
SLIMElasticNetRecommender: Processed 4583 (25.4%) in 15.01 min. Items per second: 5.09
SLIMElasticNetRecommender: Processed 6151 (34.1%) in 20.01 min. Items per second: 5.12
SLIMElasticNetRecommender: Processed 7733 (42.8%) in 25.01 min. Items per second: 5.15
SLIMElasticNetRecommender: Processed 9307 (51.5%) in 30.02 min. Items per second: 5.17
SLIMElasticNetRecommender: Processed 10902 (60.4%) in 35.02 min. Items per second: 5.19
SLIMElasticNetRecommender: Processed 12485 (69.1%) in 40.02 min. Items per second: 5.20
SLIMElasticNetRecommender: Processed 14070 (77.9%) in 45.02 min. Items per second: 5.21
SLIMElasticNetRecommender: Processed 15656 (86.7%) in 50.02 min. Items per second: 5.22
SLIMElasticNetRecommender: Processed 17226 (95.4%) in 55.02 min. Items per second: 5.22
SLIMElasticNetRecommender: Processed 18

In [36]:
recommender3 = SLIMElasticNetRecommender(URM_train)

In [37]:
recommender3.fit(l1_ratio=0.00025, alpha = 0.08, topK = 300) # 47.33%

SLIMElasticNetRecommender: Processed 1551 ( 8.6%) in 5.00 min. Items per second: 5.16
SLIMElasticNetRecommender: Processed 3142 (17.4%) in 10.00 min. Items per second: 5.23
SLIMElasticNetRecommender: Processed 4709 (26.1%) in 15.00 min. Items per second: 5.23
SLIMElasticNetRecommender: Processed 6255 (34.6%) in 20.01 min. Items per second: 5.21
SLIMElasticNetRecommender: Processed 7773 (43.0%) in 25.01 min. Items per second: 5.18
SLIMElasticNetRecommender: Processed 9321 (51.6%) in 30.01 min. Items per second: 5.18
SLIMElasticNetRecommender: Processed 10858 (60.1%) in 35.01 min. Items per second: 5.17
SLIMElasticNetRecommender: Processed 12357 (68.4%) in 40.01 min. Items per second: 5.15
SLIMElasticNetRecommender: Processed 13796 (76.4%) in 45.01 min. Items per second: 5.11
SLIMElasticNetRecommender: Processed 15340 (84.9%) in 50.02 min. Items per second: 5.11
SLIMElasticNetRecommender: Processed 16904 (93.6%) in 55.02 min. Items per second: 5.12
SLIMElasticNetRecommender: Processed 18

In [17]:
recommender2 = IALSRecommender(URM_train)

In [18]:
recommender2.fit(epochs = 32, num_factors = 50, alpha = 1.011955985422186, reg = 0.0006510527003718695, init_mean = 0.4902658701038708, init_std = 0.0542837791767091) # Così ottengo: 0.418 MAP: 0.2335376

IALSRecommender: Epoch 1 of 32. Elapsed time 13.59 sec
IALSRecommender: Epoch 2 of 32. Elapsed time 27.10 sec
IALSRecommender: Epoch 3 of 32. Elapsed time 40.78 sec
IALSRecommender: Epoch 4 of 32. Elapsed time 54.48 sec
IALSRecommender: Epoch 5 of 32. Elapsed time 1.14 min
IALSRecommender: Epoch 6 of 32. Elapsed time 1.36 min
IALSRecommender: Epoch 7 of 32. Elapsed time 1.59 min
IALSRecommender: Epoch 8 of 32. Elapsed time 1.82 min
IALSRecommender: Epoch 9 of 32. Elapsed time 2.05 min
IALSRecommender: Epoch 10 of 32. Elapsed time 2.28 min
IALSRecommender: Epoch 11 of 32. Elapsed time 2.50 min
IALSRecommender: Epoch 12 of 32. Elapsed time 2.73 min
IALSRecommender: Epoch 13 of 32. Elapsed time 2.96 min
IALSRecommender: Epoch 14 of 32. Elapsed time 3.18 min
IALSRecommender: Epoch 15 of 32. Elapsed time 3.41 min
IALSRecommender: Epoch 16 of 32. Elapsed time 3.64 min
IALSRecommender: Epoch 17 of 32. Elapsed time 3.86 min
IALSRecommender: Epoch 18 of 32. Elapsed time 4.09 min
IALSRecommender

In [19]:
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

#SLIM TOK K 3000 NORM 2

In [49]:
# 0.0
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 33.13 sec. Users per second: 412


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.382112,0.383652,0.067836,0.233438,0.234225,0.636032,0.397326,0.115218,0.96629,1.19917,0.00561429,0.490684,0.949267,0.99492,0.0672241,0.0569799,0.999707,0.966007,0.015267,8.45674,0.995304,0.0615781,0.682547,2.39514,0.0264156


In [50]:
# 0.1
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.1)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.29 sec. Users per second: 423


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.389968,0.391562,0.0691796,0.239937,0.240739,0.640768,0.40473,0.117513,0.967243,1.21945,0.00559074,0.503826,0.943292,0.994322,0.064843,0.0550972,0.999707,0.96696,0.0140603,8.3257,0.994706,0.0567107,0.671971,2.45929,0.0263048


In [51]:
# 0.2
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.2)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.34 sec. Users per second: 422


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.394848,0.396487,0.070086,0.244199,0.245019,0.646013,0.409801,0.119042,0.968709,1.23474,0.0055726,0.514317,0.937977,0.993791,0.0636248,0.0531591,0.999707,0.968425,0.0131636,8.2199,0.994174,0.0530942,0.663431,2.51049,0.0262195


In [52]:
# 0.3
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.3)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.44 sec. Users per second: 421


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.397911,0.39959,0.0707472,0.246712,0.247529,0.648754,0.412711,0.120135,0.970614,1.2428,0.00555757,0.523134,0.933085,0.993302,0.0641232,0.0522177,0.999707,0.97033,0.0124576,8.1302,0.993685,0.0502467,0.656192,2.55353,0.0261488


In [53]:
# 0.4
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.4)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.47 sec. Users per second: 420


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.400491,0.402232,0.0713586,0.248327,0.249158,0.649318,0.414765,0.121134,0.971054,1.24732,0.00554596,0.530195,0.928866,0.99288,0.0636802,0.051664,0.999707,0.970769,0.011926,8.05802,0.993263,0.0481024,0.650366,2.588,0.0260941


In [54]:
# 0.5
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.5)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.32 sec. Users per second: 422


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.401935,0.403686,0.0716291,0.249622,0.250492,0.65026,0.416177,0.12159,0.971933,1.25113,0.0055361,0.536244,0.925054,0.992499,0.0650645,0.0508888,0.999707,0.971648,0.0114965,7.99661,0.992882,0.0463701,0.645409,2.61752,0.0260478


In [55]:
# 0.6
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.6)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.46 sec. Users per second: 420


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.402946,0.404736,0.0718927,0.250141,0.251015,0.649328,0.41684,0.122016,0.97208,1.25204,0.00552827,0.541189,0.921815,0.992175,0.0663381,0.0505565,0.999707,0.971795,0.0111694,7.94685,0.992558,0.0450508,0.641394,2.64166,0.0260109


In [56]:
# 0.7
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.7)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.40 sec. Users per second: 421


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403298,0.405137,0.0720926,0.250358,0.251239,0.649596,0.417129,0.12232,0.9723,1.25272,0.00552297,0.544703,0.919408,0.991934,0.0679993,0.0502243,0.999707,0.972015,0.0109371,7.91046,0.992317,0.0441139,0.638456,2.65881,0.025986


In [60]:
# 0.75
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.75)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 36.35 sec. Users per second: 375


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403444,0.405272,0.0720107,0.250325,0.25119,0.649035,0.417026,0.122208,0.972006,1.25172,0.00552046,0.54632,0.918299,0.991823,0.0686085,0.050335,0.999707,0.971722,0.0108458,7.89497,0.992206,0.0437454,0.637206,2.66671,0.0259742


In [57]:
# 0.8
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.8)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.34 sec. Users per second: 422


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.403474,0.405291,0.0720331,0.250378,0.251231,0.649071,0.417058,0.122242,0.97208,1.25188,0.0055184,0.547716,0.917274,0.991721,0.0692729,0.0505565,0.999707,0.971795,0.01076,7.88064,0.992104,0.0433994,0.636049,2.67352,0.0259645


In [61]:
# 0.85
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.85)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 33.60 sec. Users per second: 406


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.402975,0.40478,0.071949,0.250116,0.250967,0.649743,0.416762,0.122098,0.972006,1.25163,0.00551661,0.54896,0.916395,0.991633,0.0700482,0.0501689,0.999707,0.971722,0.0106816,7.86778,0.992016,0.0430832,0.635012,2.67959,0.025956


In [58]:
# 0.9
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 0.9)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.56 sec. Users per second: 419


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.402565,0.404376,0.0719356,0.249714,0.250565,0.649609,0.41639,0.12206,0.972373,1.25059,0.00551541,0.549899,0.915656,0.991559,0.0707127,0.0501135,0.999707,0.972088,0.0106297,7.85817,0.991942,0.0428739,0.634236,2.68418,0.0259504


In [59]:
# 1
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender, recommender2)
hybrid.fit(norm = -np.inf, alpha = 1)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 32.67 sec. Users per second: 418


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.401671,0.403454,0.0718521,0.248808,0.249654,0.648586,0.415448,0.121899,0.972519,1.2477,0.00551275,0.551821,0.914229,0.991416,0.0720416,0.0506673,0.999707,0.972234,0.0105253,7.83906,0.991799,0.0424527,0.632694,2.69356,0.0259379


#SLIM TOP K 300 NORM 2

In [38]:
# 0.0
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.18 min. Users per second: 104


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.382112,0.383652,0.067836,0.233438,0.234225,0.636032,0.397326,0.115218,0.96629,1.19917,0.00561429,0.490684,0.949267,0.99492,0.0672241,0.0569799,0.999707,0.966007,0.015267,8.45674,0.995304,0.0615781,0.682547,2.39514,0.0264156


In [39]:
# 0.1
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.1)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.13 min. Users per second: 107


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.388165,0.389739,0.0688422,0.238637,0.239429,0.640984,0.40324,0.116944,0.96739,1.21611,0.00560098,0.498686,0.945258,0.994519,0.0658397,0.0559832,0.999707,0.967106,0.0145091,8.37338,0.994903,0.0585211,0.675819,2.43419,0.026353


In [40]:
# 0.2
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.2)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.05 min. Users per second: 111


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.392254,0.393896,0.0696448,0.242166,0.242977,0.643761,0.40732,0.118288,0.968269,1.22777,0.00558966,0.505895,0.941258,0.994119,0.0649538,0.0543219,0.999707,0.967985,0.0138443,8.29585,0.994503,0.0558396,0.669561,2.46939,0.0262997


In [41]:
# 0.3
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.3)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.02 min. Users per second: 112


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.395735,0.397401,0.070332,0.244893,0.245714,0.645995,0.410466,0.119437,0.969222,1.23598,0.00557926,0.512629,0.937185,0.993712,0.0650645,0.0537128,0.999707,0.968938,0.0132454,8.2222,0.994095,0.0534242,0.663617,2.50226,0.0262508


In [42]:
# 0.4
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.4)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.02 min. Users per second: 113


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.397977,0.399658,0.0708136,0.246633,0.247454,0.646728,0.412464,0.120234,0.970541,1.24104,0.00557081,0.518383,0.933396,0.993333,0.0657844,0.0535467,0.999707,0.970256,0.0127534,8.15805,0.993716,0.0514398,0.658439,2.53034,0.0262111


In [43]:
# 0.5
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.5)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.02 min. Users per second: 112


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.400286,0.40202,0.0712432,0.248463,0.249323,0.648394,0.414611,0.120958,0.970614,1.24679,0.00556369,0.523506,0.92969,0.992962,0.0665042,0.0531037,0.999707,0.97033,0.012329,8.09939,0.993346,0.049728,0.653705,2.55535,0.0261775


In [44]:
# 0.6
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.6)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.03 min. Users per second: 112


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.401033,0.402792,0.0714852,0.249091,0.249963,0.648647,0.415353,0.121341,0.971347,1.24885,0.00555818,0.527604,0.926452,0.992638,0.0674456,0.0523839,0.999707,0.971062,0.0119824,8.04985,0.993022,0.0483298,0.649707,2.57535,0.0261516


In [45]:
# 0.7
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.7)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.04 min. Users per second: 111


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.401246,0.403054,0.0715918,0.249054,0.249909,0.648828,0.415355,0.121504,0.972006,1.24837,0.00555431,0.530832,0.923723,0.992366,0.0691622,0.0523839,0.999707,0.971722,0.0117407,8.01191,0.992749,0.0473549,0.646644,2.59111,0.0261334


In [46]:
# 0.8
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.2)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.03 min. Users per second: 112


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.392254,0.393896,0.0696448,0.242166,0.242977,0.643761,0.40732,0.118288,0.968269,1.22777,0.00558966,0.505895,0.941258,0.994119,0.0649538,0.0543219,0.999707,0.967985,0.0138443,8.29585,0.994503,0.0558396,0.669561,2.46939,0.0262997


In [47]:
# 0.9
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 0.9)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.05 min. Users per second: 111


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.400227,0.402063,0.0716636,0.247888,0.248755,0.648659,0.414401,0.121561,0.972959,1.24571,0.00555072,0.534978,0.919681,0.991961,0.0732045,0.052993,0.999707,0.972674,0.0114451,7.95963,0.992344,0.0461629,0.642425,2.61134,0.0261165


In [48]:
# 1
hybrid = DifferentLossScoresHybridRecommender(URM_train, recommender3, recommender2)
hybrid.fit(norm = 2, alpha = 1)
result_df, _ = evaluator_validation.evaluateRecommender(hybrid)
result_df

EvaluatorHoldout: Processed 13646 (100.0%) in 2.36 min. Users per second: 96


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
10,0.399582,0.401398,0.0715377,0.247138,0.247995,0.646756,0.413507,0.12135,0.972813,1.24237,0.00555133,0.535755,0.918481,0.991841,0.0761393,0.0541004,0.999707,0.972527,0.0114314,7.94955,0.992224,0.0461075,0.641611,2.61514,0.0261194


#Building the final recommender

In [62]:
recSLIM = SLIMElasticNetRecommender(URM_sparse)

In [63]:
recMFIALS = IALSRecommender(URM_sparse)

In [64]:
recSLIM.fit(l1_ratio = 0.0002204448537910578, alpha = 0.079724114437381, topK = 3100)

SLIMElasticNetRecommender: Processed 1124 ( 6.2%) in 5.00 min. Items per second: 3.74
SLIMElasticNetRecommender: Processed 2321 (12.9%) in 10.00 min. Items per second: 3.87
SLIMElasticNetRecommender: Processed 3521 (19.5%) in 15.01 min. Items per second: 3.91
SLIMElasticNetRecommender: Processed 4711 (26.1%) in 20.01 min. Items per second: 3.92
SLIMElasticNetRecommender: Processed 5892 (32.6%) in 25.01 min. Items per second: 3.93
SLIMElasticNetRecommender: Processed 7081 (39.2%) in 30.01 min. Items per second: 3.93
SLIMElasticNetRecommender: Processed 8269 (45.8%) in 35.01 min. Items per second: 3.94
SLIMElasticNetRecommender: Processed 9467 (52.4%) in 40.01 min. Items per second: 3.94
SLIMElasticNetRecommender: Processed 10674 (59.1%) in 45.02 min. Items per second: 3.95
SLIMElasticNetRecommender: Processed 11868 (65.7%) in 50.02 min. Items per second: 3.95
SLIMElasticNetRecommender: Processed 13065 (72.3%) in 55.02 min. Items per second: 3.96
SLIMElasticNetRecommender: Processed 1425

In [65]:
recMFIALS.fit(epochs = 32, num_factors = 50, alpha = 1.011955985422186, reg = 0.0006510527003718695, init_mean = 0.4902658701038708, init_std = 0.0542837791767091)

IALSRecommender: Epoch 1 of 32. Elapsed time 13.08 sec
IALSRecommender: Epoch 2 of 32. Elapsed time 25.98 sec
IALSRecommender: Epoch 3 of 32. Elapsed time 38.87 sec
IALSRecommender: Epoch 4 of 32. Elapsed time 51.74 sec
IALSRecommender: Epoch 5 of 32. Elapsed time 1.08 min
IALSRecommender: Epoch 6 of 32. Elapsed time 1.29 min
IALSRecommender: Epoch 7 of 32. Elapsed time 1.51 min
IALSRecommender: Epoch 8 of 32. Elapsed time 1.72 min
IALSRecommender: Epoch 9 of 32. Elapsed time 1.94 min
IALSRecommender: Epoch 10 of 32. Elapsed time 2.15 min
IALSRecommender: Epoch 11 of 32. Elapsed time 2.37 min
IALSRecommender: Epoch 12 of 32. Elapsed time 2.59 min
IALSRecommender: Epoch 13 of 32. Elapsed time 2.80 min
IALSRecommender: Epoch 14 of 32. Elapsed time 3.02 min
IALSRecommender: Epoch 15 of 32. Elapsed time 3.24 min
IALSRecommender: Epoch 16 of 32. Elapsed time 3.45 min
IALSRecommender: Epoch 17 of 32. Elapsed time 3.67 min
IALSRecommender: Epoch 18 of 32. Elapsed time 3.89 min
IALSRecommender

In [76]:
hybrid_SLIM_MFIALS = DifferentLossScoresHybridRecommender(URM_sparse, recSLIM, recMFIALS)
hybrid_SLIM_MFIALS.fit(norm = -np.inf, alpha = 0.8)

In [77]:
%cd RSChallenge/
%ls

/content/gdrive/MyDrive/Colab Notebooks/RSChallenge
HYBRID.ipynb  Results/  RSData/  RSLib/  RSLib2/  submission_HYBRID_1.csv


In [78]:
target_users = pd.read_csv('RSData/data_target_users_test.csv')

In [79]:
final_dict = {}
start_time = time.time()

for user in target_users['user_id']:
  final_dict[user] = hybrid_SLIM_MFIALS.recommend(user,cutoff=10)
end_time = time.time()
print('Time needed is {}seconds'.format(end_time-start_time))

Time needed is 50.41395306587219seconds


In [80]:
item_list = []
for user in target_users['user_id']:
  item_list.append(' '.join(str(e) for e in final_dict[user]))

In [81]:
submission_df = pd.DataFrame({'user_id' : target_users['user_id'],'item_list' : item_list })
submission_df['item_list'] = submission_df['item_list'].astype(str).str.strip('[]')
submission_df

,user_id,item_list
0,0,3899 5433 16365 8465 16359 2243 6061 9130 640 ...
1,1,3473 940 14400 16012 2118 640 9275 16365 10118...
2,2,6177 1551 17022 17330 14229 16365 2846 15119 4...
3,3,8194 15551 15082 16359 8371 960 15507 16444 58...
4,4,8194 10586 6177 2152 9871 10954 5299 6011 1649...
...,...,...
13645,13645,6177 11491 3200 8898 1259 11576 7762 11303 407...
13646,13646,3473 7867 8635 16640 17266 17623 409 11504 176...
13647,13647,1846 7716 7606 11227 8635 14868 1186 11491 168...
13648,13648,6177 8635 13295 6214 7295 14358 14181 17936 52...


In [82]:
%ls
%cd ..

HYBRID.ipynb  Results/  RSData/  RSLib/  RSLib2/  submission_HYBRID_1.csv
/content/gdrive/My Drive/Colab Notebooks


In [83]:
submission_df.to_csv('RSChallenge/submission_HYBRID_2.csv', index=False)